# Exercise 2.2 - Random Matrix theory

In [1]:
import numpy as np
import networkx as nx

import disordered_sys.generate

In [6]:
epsilon = 1e-3
c = 3
n = 2**11

m, g = disordered_sys.generate.generate_m(c, n, return_g=True)

np.array(list(g.neighbors(3)))

k = 1
j = g.neighbors(k)[1]
set(g.neighbors(k)) - {j}

array([327, 962, 644])

In [3]:
def cavity_equation(neighbors, lam):
    epsilon + 1j * lam + 1 / neighbour_prec